In [1]:
from encoder_paths import *
import json
from pprint import pprint
TRAIN_FILE_PATH = "/tmp/semeval24_task3/SemEval-2024_Task3/official_data/Training_data/text/training.json"
VALIDATION_FILE_PATH = "/tmp/semeval24_task3/SemEval-2024_Task3/official_data/Training_data/text/testing.json"
with open(TRAIN_FILE_PATH) as f:
    train_data = json.load(f)
with open(VALIDATION_FILE_PATH) as f:
    validation_data = json.load(f)

pprint(len(train_data))
pprint(len(validation_data))

1236
138


In [2]:
class EmotionIndexer:
    def __init__(self):
        self.emotion_to_index = {
            'joy': 0,
            'sadness': 1,
            'anger': 2,
            'neutral': 3,
            'surprise': 4,
            'disgust': 5,
            'fear': 6,
            'pad': 7,
        }
        self.emotion_freq = [0]*7
        self.weights = None

        self.index_to_emotion = {index: emotion for emotion, index in self.emotion_to_index.items()}

    def emotion_to_idx(self, emotion):
        return self.emotion_to_index.get(emotion, None)

    def idx_to_emotion(self, index):
        return self.index_to_emotion.get(index, None)
    
    def compute_weights(self, data):
        for conversation in data:
            conversation = conversation['conversation']
            for utterance in conversation:
                emotion = utterance['emotion']
                self.emotion_freq[self.emotion_to_index[emotion]] += 1
        print(self.emotion_freq)
        self.weights = [1/freq for freq in self.emotion_freq]

# Example usage
indexer = EmotionIndexer()
indexer.compute_weights(train_data)
print(indexer.weights)

[2059, 1024, 1472, 5282, 1647, 369, 326]
[0.00048567265662943174, 0.0009765625, 0.0006793478260869565, 0.0001893222264293828, 0.0006071645415907711, 0.0027100271002710027, 0.003067484662576687]


In [3]:
all_emotions = set()
for conversation in train_data:
    conversation = conversation["conversation"]
    for utterance in conversation:
        all_emotions.add(utterance["emotion"])
pprint(all_emotions)

{'joy', 'disgust', 'fear', 'sadness', 'anger', 'surprise', 'neutral'}


In [4]:
import torch
import json
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_video
from torchvision.transforms import functional as F
from PIL import Image
import numpy as np

/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# video_id_mapping = np.load("/home2/akshett.jindal/research/semeval24/task3/MECPE/data/video_id_mapping.npy", allow_pickle=True)
# print(video_id_mapping)
# video_id_mapping = video_id_mapping.item()

In [6]:
import torch
import pickle

class YourAudioEncoder():
    def __init__(self, audio_embeddings_path):
        with open(audio_embeddings_path, "rb") as f:
            self.audio_embeddings = pickle.load(f)

    def lmao(self, audio_name):
        audio_name = audio_name.split(".")[0]
        audio_embedding = self.audio_embeddings[audio_name]
        audio_embedding = audio_embedding.squeeze()
        return torch.from_numpy(audio_embedding)
    
class YourVideoEncoder():
    def __init__(self, video_embeddings_path):
        with open(video_embeddings_path, "rb") as f:
            self.video_embeddings = pickle.load(f)

    def lmao(self, video_name):
        # video_name = video_name.split(".")[0]
        video_embedding = self.video_embeddings[video_name].reshape((16,-1))
        video_embedding = np.mean(video_embedding, axis=0)
        return torch.from_numpy(video_embedding)

class YourTextEncoder():
    def __init__(self, text_embeddings_path):
        with open(text_embeddings_path, "rb") as f:
            self.text_embeddings = pickle.load(f)

    def lmao(self, video_name):
        text_embedding = self.text_embeddings[video_name]
        return torch.from_numpy(text_embedding)


In [7]:
class ConversationDataset(Dataset):
    def __init__(self, json_file, audio_encoder, video_encoder, text_encoder, max_seq_len):
        self.max_seq_len = max_seq_len
        self.data = self.load_data(json_file)
        self.audio_encoder = audio_encoder
        self.video_encoder = video_encoder
        self.text_encoder = text_encoder

    def load_data(self, json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        conversation = self.data[idx]['conversation']
        emotion_labels = [utterance['emotion'] for utterance in conversation]
        audio_paths = [utterance['video_name'].replace('mp4', 'wav') for utterance in conversation]
        video_paths = [utterance['video_name'] for utterance in conversation]
        texts = [utterance['video_name'] for utterance in conversation]

        audio_embeddings = [self.audio_encoder.lmao(audio_path) for audio_path in audio_paths]
        video_embeddings = [self.video_encoder.lmao(video_path) for video_path in video_paths]
        text_embeddings = [self.text_encoder.lmao(text) for text in texts]

        cause_pairs = self.data[idx]['emotion-cause_pairs']
        useful_utterances = set([int(cause_pair[1]) for cause_pair in cause_pairs])
        cause_labels = []
        for utterance in conversation:
            if utterance['utterance_ID'] in useful_utterances:
                cause_labels.append(1)
            else:
                cause_labels.append(0)
        
        
        # Pad or truncate conversations to the maximum sequence length
        if len(conversation) < self.max_seq_len and False:
            pad_length = self.max_seq_len - len(conversation)
            audio_embeddings += [torch.zeros_like(audio_embeddings[0])] * pad_length
            video_embeddings += [torch.zeros_like(video_embeddings[0])] * pad_length
            text_embeddings += [torch.zeros_like(text_embeddings[0])] * pad_length
            cause_labels += [-1] * pad_length
            emotion_labels += ['pad'] * pad_length
            pad_mask = [1] * len(conversation) + [0] * pad_length
        else:
            audio_embeddings = audio_embeddings[:self.max_seq_len]
            video_embeddings = video_embeddings[:self.max_seq_len]
            text_embeddings = text_embeddings[:self.max_seq_len]
            emotion_labels = emotion_labels[:self.max_seq_len]
            cause_labels = cause_labels[:self.max_seq_len]
            pad_mask = [1] * self.max_seq_len

        emotion_indices = [indexer.emotion_to_idx(emotion) for emotion in emotion_labels]
        
        audio_embeddings = torch.stack(audio_embeddings)
        video_embeddings = torch.stack(video_embeddings)
        text_embeddings = torch.stack(text_embeddings)
        emotion_indices = torch.from_numpy(np.array(emotion_indices))
        pad_mask = torch.from_numpy(np.array(pad_mask))
        cause_labels = torch.from_numpy(np.array(cause_labels))
        
        return {
            'audio': audio_embeddings,
            'video': video_embeddings,
            'text': text_embeddings,
            'emotion_labels': emotion_indices,
            'pad_mask': pad_mask,
            'cause_labels': cause_labels,
        }
# Example usage
# You need to define your audio, video, and text encoders accordingly

# Define your data paths
# DATA_DIR = "/tmp/semeval24_task3"

# AUDIO_EMBEDDINGS_FILEPATH = os.path.join(DATA_DIR, "audio_embeddings", "audio_embeddings.pkl")
# VIDEO_EMBEDDINGS_FILEPATH = os.path.join(DATA_DIR, "video_embeddings", "train", "video_embeddings.pkl")
# TEXT_EMBEDDINGS_FILEPATH = os.path.join(DATA_DIR, "text_embeddings", "text_embeddings.pkl")

audio_encoder = YourAudioEncoder(AUDIO_EMBEDDINGS_FILEPATH)
video_encoder = YourVideoEncoder(VIDEO_EMBEDDINGS_FILEPATH)
text_encoder = YourTextEncoder(TEXT_EMBEDDINGS_FILEPATH)
max_seq_len = 40  # Adjust this according to your needs

# Create the dataset and dataloader
train_dataset = ConversationDataset(TRAIN_FILE_PATH, audio_encoder, video_encoder, text_encoder, max_seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

validation_dataset = ConversationDataset(VALIDATION_FILE_PATH, audio_encoder, video_encoder, text_encoder, max_seq_len)
validation_dataloader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

# Example of iterating through batches
# for batch in dataloader:
#     audio = batch['audio']  # Shape: (batch_size, max_seq_len, audio_embedding_size)
#     video = batch['video']  # Shape: (batch_size, max_seq_len, video_embedding_size)
#     text = batch['text']    # Shape: (batch_size, max_seq_len, text_embedding_size)
#     emotions = batch['emotion_labels']  # List of emotion labels for each utterance in the batch


In [8]:
# import torch
# import torch.nn as nn
# from torch.nn import TransformerEncoder, TransformerEncoderLayer

# class EmotionClassifier(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout, num_emotions):
#         super(EmotionClassifier, self).__init__()
        
#         self.first_linear = nn.Linear(input_size, hidden_size, dtype=torch.float32)

#         self.transformer_encoder = TransformerEncoder(
#             TransformerEncoderLayer(hidden_size, num_heads, hidden_size, dropout),
#             num_layers
#         )
        
#         self.linear = nn.Linear(hidden_size, num_emotions)

#     def forward(self, audio_encoding, video_encoding, text_encoding):

#         # Concatenate or combine the audio, video, and text encodings here
#         # You can use any method like concatenation, addition, or other fusion techniques
#         # Combine the encodings (you can customize this part)
#         audio_encoding = audio_encoding.float()
#         video_encoding = video_encoding.float()
#         text_encoding = text_encoding.float().squeeze()
#         combined_encoding = torch.cat((audio_encoding, video_encoding, text_encoding), dim=2)
        
#         combined_encoding = self.first_linear(combined_encoding)
        
        
#         combined_encoding = combined_encoding.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, input_size)
        
        
#         transformer_output = self.transformer_encoder(combined_encoding)

#         # Take the output of the Transformer encoder for the last position as the summary
#         emotion_logits = self.linear(transformer_output.permute(1, 0, 2))
#         # Apply a softmax layer
#         emotion_logits = torch.softmax(emotion_logits, dim=2)

#         return emotion_logits

In [9]:
import torch
import torch.nn as nn
from TorchCRF import CRF

class EmotionClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, num_emotions, embedding_dropout=0.2):
        super(EmotionClassifier, self).__init__()
        
        self.audio_dropout = nn.Dropout(embedding_dropout)
        self.video_dropout = nn.Dropout(embedding_dropout)
        self.text_dropout = nn.Dropout(embedding_dropout)

        self.first_linear = nn.Linear(input_size, hidden_size, dtype=torch.float32)
        self.relu = nn.ReLU()
        
        self.second_linear_layer = nn.Linear(hidden_size, hidden_size, dtype=torch.float32)
        # Replace Transformer with BiLSTM
        self.bilstm = nn.LSTM(hidden_size, hidden_size // 2, num_layers, 
                              dropout=dropout, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(hidden_size, num_emotions)
        self.crf_model = CRF(num_emotions)
        

    def generate_emissions(self, audio_encoding, video_encoding, text_encoding):
        # Concatenate or combine the audio, video, and text encodings
        audio_encoding = audio_encoding.float()
        video_encoding = video_encoding.float()
        text_encoding = text_encoding.float()
        
        audio_encoding = self.audio_dropout(audio_encoding)
        video_encoding = self.video_dropout(video_encoding)
        text_encoding = self.text_dropout(text_encoding)
        
        combined_encoding = torch.cat((audio_encoding, video_encoding, text_encoding), dim=2)
        
        combined_encoding = self.first_linear(combined_encoding)
        combined_encoding = self.relu(combined_encoding)
        combined_encoding = self.second_linear_layer(combined_encoding)
        
        # Pass through BiLSTM
        lstm_output, _ = self.bilstm(combined_encoding)

        # Take the output of the BiLSTM
        emotion_logits = self.linear(lstm_output)
        # Apply a softmax layer
        # emotion_logits = torch.softmax(emotion_logits, dim=2)

        return emotion_logits

    def loss(self, audio_encoding, video_encoding, text_encoding, emotion_labels, padding):

        emissions = self.generate_emissions(audio_encoding, video_encoding, text_encoding)
        emotion_labels = emotion_labels.unsqueeze(1)
        x, y, _ = emissions.shape
        padding = torch.ones((x, y), dtype=torch.bool).to('cuda')
        emotion_labels = emotion_labels.squeeze(1)
        loss = -self.crf_model(emissions, emotion_labels, padding)

        return loss
    
    def predict(self, audio_encoding, video_encoding, text_encoding):
        emissions = self.generate_emissions(audio_encoding, video_encoding, text_encoding)
        x, y, _ = emissions.shape
        padding = torch.ones((x, y), dtype=torch.bool).to('cuda')
        label = self.crf_model.viterbi_decode(emissions, padding)
        return label
    
    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim).to('cuda'), torch.randn(2, 1, self.hidden_dim).to('cuda'))

In [10]:
from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F


class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.

    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.

    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.

        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v!r}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


# def focal_loss(alpha: Optional[Sequence] = None,
#                gamma: float = 0.,
#                reduction: str = 'mean',
#                ignore_index: int = -100,
#                device='cpu',
#                dtype=torch.float32) -> FocalLoss:
#     """Factory function for FocalLoss.

#     Args:
#         alpha (Sequence, optional): Weights for each class. Will be converted
#             to a Tensor if not None. Defaults to None.
#         gamma (float, optional): A constant, as described in the paper.
#             Defaults to 0.
#         reduction (str, optional): 'mean', 'sum' or 'none'.
#             Defaults to 'mean'.
#         ignore_index (int, optional): class label to ignore.
#             Defaults to -100.
#         device (str, optional): Device to move alpha to. Defaults to 'cpu'.
#         dtype (torch.dtype, optional): dtype to cast alpha to.
#             Defaults to torch.float32.

#     Returns:
#         A FocalLoss object
#     """
#     if alpha is not None:
#         if not isinstance(alpha, Tensor):
#             alpha = torch.tensor(alpha)
#         alpha = alpha.to(device=device, dtype=dtype)

#     fl = FocalLoss(
#         alpha=alpha,
#         gamma=gamma,
#         reduction=reduction,
#         ignore_index=ignore_index)
#     return fl

In [11]:
# # Define Focal Loss
# import torch.nn.functional as F

# class FocalLoss(nn.Module):
#     def __init__(self, gamma=2.0, reduction='mean'):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.reduction = reduction

#     def forward(self, input, target):
#         # Calculate Cross-Entropy Loss, but do not reduce (keep the same shape as input)
#         ce_loss = F.cross_entropy(input, target, reduction='none') 
#         pt = torch.exp(-ce_loss)  # Calculate p_t
#         focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()  # Calculate Focal Loss

#         if self.reduction == 'mean':
#             return focal_loss.mean()
#         elif self.reduction == 'sum':
#             return focal_loss.sum()
#         else:
#             return focal_loss

# # Replace the loss function with Focal Loss

In [12]:
print(indexer.emotion_freq)

[2059, 1024, 1472, 5282, 1647, 369, 326]


In [13]:
print(indexer.emotion_to_index)
# 1 1 1 0.25 1 3 3

{'joy': 0, 'sadness': 1, 'anger': 2, 'neutral': 3, 'surprise': 4, 'disgust': 5, 'fear': 6, 'pad': 7}


In [15]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import classification_report
from transformers import get_linear_schedule_with_warmup

# Define your model
# model = EmotionClassifier(input_size=11237, hidden_size=5000, num_layers=2, num_heads=2, dropout=0.2, num_emotions=7)
print(TEXT_EMBEDDINGS_FILEPATH)
print(AUDIO_EMBEDDINGS_FILEPATH)
print(VIDEO_EMBEDDINGS_FILEPATH)
model = EmotionClassifier(input_size=768*3, hidden_size=2000, num_emotions=7, num_layers=4, dropout=0.3)
model.to("cuda")

# Define your loss function and optimizer
weights_tensor = torch.tensor(np.array(indexer.weights)).to("cuda").float()
# weights_tensor = torch.tensor(np.array([1.0, 1.0, 1.0, 0.25, 1.0, 3.0, 3.0])).to("cuda").float()
criterion = nn.CrossEntropyLoss(
    weight=weights_tensor,
    ignore_index=7
)

num_epochs = 60

optimizer = AdamW(model.parameters(), lr=1e-4)

total_steps = len(train_dataloader) * num_epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

best_model_file = None
best_val_loss = float('inf')
best_epoch = -1
best_classification_report = None


# criterion = FocalLoss(gamma=2.0)

# Define training parameters

# Training loop
for epoch in (range(num_epochs)):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    total_tokens = 0
    total_correct = 0
    total_predictions = 0

    for batch in tqdm(train_dataloader):  # Assuming you have a DataLoader for your dataset
        # Extract data from the batch
        audio = batch['audio'].to('cuda')
        video = batch['video'].to('cuda')
        text = batch['text'].to('cuda')
        emotion_indices = batch['emotion_labels'].to('cuda')
        pad_mask = batch['pad_mask'].to('cuda')

        # Forward pass
        # emotion_logits = emotion_model(audio_copy, video_copy, text_copy)

        # Reshape emotion_logits
        # emotion_logits = emotion_logits.view(-1, emotion_logits.size(-1))

        # Flatten emotion_indices (assuming it's a 2D tensor with shape [batch_size, max_sequence_length])
        # emotion_indices = emotion_indices.view(-1)

        # Calculate a mask to exclude padded positions from the loss
        pad_mask = pad_mask.view(-1).float()

        # Calculate the loss, excluding padded positions
        emotion_loss = model.loss(audio, video, text, emotion_indices, pad_mask)
        # masked_loss = torch.sum(loss * pad_mask) / torch.sum(pad_mask)
        masked_loss = emotion_loss# *pad_mask
        # Backpropagation and optimization
        
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        masked_loss.backward()
        optimizer.step()
        

        total_loss += masked_loss.item()
        total_tokens += torch.sum(pad_mask).item()
        
        predicted_emotions = model.predict(audio, video, text)
        correct_predictions = ((predicted_emotions == emotion_indices) * pad_mask).sum().item()

        total_correct += correct_predictions
        total_predictions += torch.sum(pad_mask).item()  # Batch size
    
    scheduler.step()

    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0
    total_val_tokens = 0
    total_val_correct = 0
    total_val_predictions = 0
    true_labels = []
    predicted_labels = []
    padded_labels = []

    with torch.no_grad():
        for val_batch in tqdm(validation_dataloader):
            audio = val_batch['audio'].to('cuda')
            video = val_batch['video'].to('cuda')
            text = val_batch['text'].to('cuda')
            emotion_indices = val_batch['emotion_labels'].to('cuda')
            pad_mask = val_batch['pad_mask'].to('cuda')
            
            pad_mask = pad_mask.view(-1)   
            # Calculate the loss, excluding padded positions
            val_loss = model.loss(audio, video, text, emotion_indices, pad_mask)
            masked_loss = torch.sum(val_loss * pad_mask) / torch.sum(pad_mask)
            
            total_val_loss += masked_loss.item()
            total_val_tokens += torch.sum(pad_mask).item()
            
            predicted_emotions_val = model.predict(audio, video, text)
            correct_predictions_val = ((predicted_emotions_val == emotion_indices) * pad_mask).sum().item()
            total_val_correct += correct_predictions_val
            total_val_predictions += torch.sum(pad_mask).item()

            # Store true and predicted labels for F1 score calculation
            emotion_indices = emotion_indices.cpu().squeeze(0).numpy()
            true_labels.extend(emotion_indices)
            predicted_labels.extend(predicted_emotions_val[0])
            padded_labels.extend(pad_mask.cpu().numpy())

    final_true_labels = [label for label, pad in zip(true_labels, padded_labels) if pad == 1]
    final_predicted_labels = [label for label, pad in zip(predicted_labels, padded_labels) if pad == 1]
    # print("=========================")
    # print(final_true_labels)
    # print(final_predicted_labels)
    # print("=========================")
    classification_rep = classification_report(final_true_labels, final_predicted_labels)
    
    if total_val_loss < best_val_loss:
        best_val_loss = total_val_loss
        best_epoch = epoch
        best_classification_report = classification_rep
        best_model_file = f"/tmp/semeval24_task3/baseline_models/emotion_models/best_model.pt"
        torch.save(model.state_dict(), best_model_file)

    # Calculate and print the average loss for this epoch
    avg_loss = total_loss / total_tokens
    avg_val_loss = total_val_loss / total_val_tokens
    print(f"Epoch [{epoch + 1}/{num_epochs}] Training Loss: {avg_loss}")
    print(f"Epoch [{epoch + 1}/{num_epochs}] Validation Loss: {avg_val_loss}")
    print(classification_rep)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Accuracy: {total_correct / total_predictions:.4f}")

    torch.save(model.state_dict(), f"/tmp/semeval24_task3/baseline_models/emotion_models/emotion_model_{epoch:02}.pt")

print("Training complete!")
print("=========================BEST MODEL DETAILS=========================")
print(f"Best model at epoch {best_epoch + 1} with validation loss: {best_val_loss}")
print("Classification Report\n", best_classification_report)
print(f"Best model saved at: {best_model_file}")

/tmp/semeval24_task3/text_embeddings/text_embeddings_roberta_base_emotion.pkl
/tmp/semeval24_task3/audio_embeddings/audio_embeddings_microsoft_wavlm-base-plus-sd.pkl
/tmp/semeval24_task3/video_embeddings/final_embeddings.pkl


100%|██████████| 138/138 [00:01<00:00, 79.17it/s]
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parame

Epoch [1/60] Training Loss: 0.3761968681606853
Epoch [1/60] Validation Loss: 0.36512321771486944
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       242
           1       0.29      0.57      0.38       123
           2       0.00      0.00      0.00       143
           3       0.56      0.79      0.66       647
           4       0.44      0.65      0.53       193
           5       0.00      0.00      0.00        45
           6       0.00      0.00      0.00        47

    accuracy                           0.49      1440
   macro avg       0.18      0.29      0.22      1440
weighted avg       0.34      0.49      0.40      1440

Epoch [1/60] Accuracy: 0.0000


100%|██████████| 138/138 [00:01<00:00, 79.07it/s]
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home2/suyash.mathur/anaconda3/envs/mindeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parame

Epoch [2/60] Training Loss: 0.3414623186763822
Epoch [2/60] Validation Loss: 0.3427234321616698
              precision    recall  f1-score   support

           0       0.66      0.38      0.48       242
           1       0.00      0.00      0.00       123
           2       0.21      0.30      0.25       143
           3       0.59      0.80      0.68       647
           4       0.50      0.57      0.53       193
           5       0.00      0.00      0.00        45
           6       0.00      0.00      0.00        47

    accuracy                           0.53      1440
   macro avg       0.28      0.29      0.28      1440
weighted avg       0.46      0.53      0.48      1440

Epoch [2/60] Accuracy: 0.0000


...

Epoch [58/60] Training Loss: 0.00903950504307608
Epoch [58/60] Validation Loss: 0.8870307347912719
              precision    recall  f1-score   support

           0       0.53      0.50      0.52       242
           1       0.39      0.46      0.42       123
           2       0.30      0.41      0.35       143
           3       0.65      0.61      0.63       647
           4       0.51      0.56      0.53       193
           5       0.05      0.02      0.03        45
           6       0.00      0.00      0.00        47

    accuracy                           0.52      1440
   macro avg       0.35      0.37      0.35      1440
weighted avg       0.51      0.52      0.51      1440

Epoch [58/60] Accuracy: 0.0000


100%|██████████| 138/138 [00:01<00:00, 79.22it/s]


Epoch [59/60] Training Loss: 0.011876105191638169
Epoch [59/60] Validation Loss: 0.8911981464728065
              precision    recall  f1-score   support

           0       0.50      0.45      0.48       242
           1       0.37      0.37      0.37       123
           2       0.34      0.43      0.38       143
           3       0.64      0.65      0.65       647
           4       0.50      0.55      0.53       193
           5       0.13      0.07      0.09        45
           6       0.06      0.02      0.03        47

    accuracy                           0.52      1440
   macro avg       0.36      0.37      0.36      1440
weighted avg       0.51      0.52      0.51      1440

Epoch [59/60] Accuracy: 0.0000


100%|██████████| 138/138 [00:01<00:00, 80.02it/s]


Epoch [60/60] Training Loss: 0.009797847097359815
Epoch [60/60] Validation Loss: 0.9052893411422122
              precision    recall  f1-score   support

           0       0.49      0.44      0.46       242
           1       0.38      0.37      0.37       123
           2       0.37      0.31      0.34       143
           3       0.63      0.71      0.67       647
           4       0.53      0.59      0.55       193
           5       0.16      0.09      0.11        45
           6       0.12      0.04      0.06        47

    accuracy                           0.54      1440
   macro avg       0.38      0.36      0.37      1440
weighted avg       0.51      0.54      0.52      1440

Epoch [60/60] Accuracy: 0.0000
Training complete!
=========================BEST MODEL DETAILS=========================
Best model at epoch 6 with validation loss: 1768.8465381860733
Classification Report
               precision    recall  f1-score   support

           0       0.54      0.61      0.57